# ESRGAN Upscaling

This enhances the STL-10 test set by 4x going from 96x96 to 384x384, and is far superior to regular interpolation algorithms.

In [2]:
import os
import time
from PIL import Image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt

In [3]:
test_dir = 'test' #Test set directory
target_dir = 'processed_test' #New directory to save processed test images

try:
    os.mkdir(target_dir)
except: pass
file_list = []
for root, dirs, files in os.walk(test_dir, topdown=False):
    for name in files:
        file_list.append(os.path.join(root, name).replace('\\','/'))
        
SAVED_MODEL_PATH = "https://tfhub.dev/captain-pool/esrgan-tf2/1"
model = hub.load(SAVED_MODEL_PATH)        

In [4]:
def preprocess_image(image_path):
    """ Loads image from path and preprocesses to make it model ready
      Args:
        image_path: Path to the image file
    """
    hr_image = tf.image.decode_image(tf.io.read_file(image_path))
    # If PNG, remove the alpha channel. The model only supports
    # images with 3 color channels.
    if hr_image.shape[-1] == 4:
        hr_image = hr_image[...,:-1]
    hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 4) * 4
    hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
    hr_image = tf.cast(hr_image, tf.float32)
    return tf.expand_dims(hr_image, 0)

def save_image(image, filename,target_dir):
    """
    Saves unscaled Tensor Images.
    Args:
      image: 3D image tensor. [height, width, channels]
      filename: Name of the file to save to.
    """
    if not isinstance(image, Image.Image):
        image = tf.clip_by_value(image, 0, 255)
        image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
    image.save(f"{target_dir}{filename.replace(test_dir,'')}")

In [ ]:
IMAGE_PATH = 'nric.png'
hr_image = preprocess_image(IMAGE_PATH)
fake_image = model(hr_image)
fake_image = tf.squeeze(fake_image)
save_image(fake_image, filename='nric_new.png',target_dir='')

In [4]:
for file in file_list:
    IMAGE_PATH = file
    hr_image = preprocess_image(IMAGE_PATH)
    fake_image = model(hr_image)
    fake_image = tf.squeeze(fake_image)
    save_image(fake_image, filename=file,target_dir)